In [23]:
from pymongo import MongoClient
import time
import operator
import datetime
import re

client = MongoClient()
db = client.twitter
te = db['prueba2']
dia_inicio = datetime.datetime.now()
dia_inicio = datetime.datetime(dia_inicio.year, dia_inicio.month, 25)
#dia_inicio = te.find_one(sort=[("created_at", 1)])["created_at"]
print dia_inicio

#fin = te.find_one(sort=[("created_at", -1)])["created_at"]
fin = datetime.datetime(dia_inicio.year, dia_inicio.month, 26)
print fin

2017-01-25 00:00:00
2017-01-26 00:00:00


In [5]:
def insert_web_dict(d,value):
    if not value:
        return
    if 'https://twitter.com/' in value:
        return
    if value in d:
        d[value] = d[value] + 1
    else:
        d[value] = 1


        
    

In [16]:
def get_trending(init, end, limit):
    dw = {}
    count = 0
    for a in te.find({'$and': [{'created_at': {"$gte" : init}} ,{'created_at': {"$lt" : end}} ]}):
        count = count +1
        if len(a['entities']['urls']) >0:
            for b in a['entities']['urls']:
                insert_web_dict(dw,b['expanded_url'])
        if 'retweeted_status' in a and len(a['retweeted_status']['entities']['urls']) >0:      
            for b in a['retweeted_status']['entities']['urls']:
                insert_web_dict(dw,b['expanded_url'])
    trending = sorted(dw.items(), key=operator.itemgetter(1), reverse=True)
    return trending[:limit], count

In [27]:
intervalo = 60
a =  datetime.datetime(dia_inicio.year, dia_inicio.month, dia_inicio.day, dia_inicio.hour, dia_inicio.minute)
while fin > a:
    b = a + datetime.timedelta(minutes=intervalo)
    (tr, count)= get_trending(a, b, 5)
    if count < 3000:
        tr = []
    first = True
    for w in tr:
        if w:
            if first:
                print str(a) + ':'
                first = False
            print '\t' + str(w)
    a = b

2017-01-25 08:00:00:
	(u'http://www.elmundo.es/espana/2017/01/25/5887bf88e2704e8e6c8b4661.html', 301)
	(u'http://www.elmundotoday.com/2017/01/el-vaticano-admite-que-algunos-sacerdotes-se-dopan-para-tener-mas-fe', 196)
	(u'http://www.elmundo.es/espana/2017/01/25/5887bb53e2704e23158b4628.html', 135)
	(u'http://elconfidencial.com', 122)
	(u'http://prnoticias.com/internet-y-redes-sociales/diarios-digitales/20159751-comscore-diciembre-batacazo-de-elconfidencial-com', 122)
2017-01-25 09:00:00:
	(u'http://www.elmundo.es/espana/2017/01/25/5887bf88e2704e8e6c8b4661.html', 263)
	(u'http://www.elmundotoday.com/2017/01/el-vaticano-admite-que-algunos-sacerdotes-se-dopan-para-tener-mas-fe', 165)
	(u'http://elconfidencial.com', 116)
	(u'http://prnoticias.com/internet-y-redes-sociales/diarios-digitales/20159751-comscore-diciembre-batacazo-de-elconfidencial-com', 116)
	(u'http://www.elmundo.es/espana/2017/01/25/5887bb53e2704e23158b4628.html', 110)
2017-01-25 10:00:00:
	(u'http://www.elmundotoday.com/201

In [93]:
###Creamos rangos y cogemos lo más visitado
d = {}
for a in te.find({'$and': [{'text':{"$regex" : '#'}} ,{'created_at': {"$gt" : dia_inicio}} ]},  { 'text': 1}):
	s = a['text']
	for ht in  [b.strip('#') for b in s.split() if b.startswith("#")]:
		if ht not in d:
			d[ht] = 1
		else:
			d[ht] = d[ht] + 1
trending = sorted(d.items(), key=operator.itemgetter(1), reverse=True)
for a in trending:
    print a

(u'Trump', 17)
(u'aborto', 16)
(u'globalizaci\xf3n', 12)
(u'CAN2017', 7)
(u'Espa\xf1a', 6)
(u'ultimahora', 5)
(u'TPP', 4)
(u'Columnista', 4)
(u'PremiosTVyNovelas2017', 4)
(u'elmundo', 3)
(u'aut\xf3nomos', 3)
(u'BoycottCesar', 3)
(u'TurismoMadridFITUR', 3)
(u'KnowYourLemons', 3)
(u'Italia', 3)
(u'Redes', 3)
(u'Odebrecht,', 3)
(u'ElEconomista', 3)
(u'CmDayJaen', 3)
(u'ConoceTusLimones', 3)
(u'WomenMarch', 3)
(u'Fotogaler\xeda', 3)
(u'NOW2016', 3)
(u'NOWOneDirection', 3)
(u'juventud', 3)
(u'Economia', 3)
(u'ElPaisDeporte', 3)
(u'avalancha', 2)
(u'AusOpen', 2)
(u'Marca', 2)
(u'ElMundo', 2)
(u'soyespectador', 2)
(u'digital', 2)
(u'Jerusal\xe9n', 2)
(u'tr\xe1ficodearmas', 2)
(u'ElPais', 2)
(u'NoticiasEspa\xf1a', 2)
(u'Gales', 2)
(u'Huracan', 2)
(u'MarcaPersonal', 2)
(u'Primarias', 2)
(u'Ecopetrol', 2)
(u'nba', 2)
(u'Francia', 2)
(u'HuesoDuroDRoerPruebenComoLsAjustesEnEspa\xf1a', 2)
(u'ejecuciones', 2)
(u'oleoducto', 2)
(u'limones', 2)
(u'Constituci\xf3n', 2)
(u'Universidad', 2)
(u'Adictosala

In [119]:
dw2 = {}
for a in te.find():
    lh = []
    for b in a['entities']['hashtags']:
        lh.append(b['text'])
    if 'retweeted_status' in a:   
        for b in a['retweeted_status']['entities']['hashtags']:
            lh.append(b['text'])
    
    
    
    
    if len(a['entities']['urls']) >0:
        for b in a['entities']['urls']:
            if b['expanded_url'] in dw2:
                dw2[b['expanded_url']] = dw2[b['expanded_url']] + lh
            else:
                dw2[b['expanded_url']] = lh
    if 'retweeted_status' in a and len(a['retweeted_status']['entities']['urls']) >0:      
        for b in a['retweeted_status']['entities']['urls']:
            if b['expanded_url'] in dw2:
                dw2[b['expanded_url']] = dw2[b['expanded_url']] + lh
            else:
                dw2[b['expanded_url']] = lh
            
trending = sorted(dw.items(), key=operator.itemgetter(1), reverse=True)
i = 1

   
for a in trending:
    print a
    print set(dw2[a[0]])
    i = i+1
    if i > limit:
        break

(u'http://elpais.com/elpais/2017/01/23/opinion/1485196105_786881.html', 1094)
set([u'abogadosdeatocha', u'Democracia', u'Izquierda', u'Portada', u'AbogadosDeAtocha', u'40aniversario', u'democracia', u'FelizMartes', u'Atocha', u'AbogadosdeAtocha', u'libertad'])
(u'http://www.elmundotoday.com/2017/01/u2-hara-un-concierto-acustico-debido-al-precio-de-la-luz', 1056)
set([])
(u'http://www.elmundotoday.com/2017/01/15-predicciones-que-hicieron-los-fruitis', 459)
set([])
(u'http://economia.elpais.com/economia/2017/01/15/actualidad/1484514194_176496.html?id_externo_rsoc=TW_CC', 417)
set([u'cfd1617', u'innovacion', u'innovaci\xf3n', u'edu', u'Deveuenveu', u'education', u'PeterSenge', u'TIC', u'Educaci\xf3n', u'eftrassierra', u'edtech', u'formacion', u'edchat', u'Pedagog\xeda', u'educaci\xf3n', u'iestrassierra', u'SoL', u'profesor', u'aprendizaje', u'rrhh', u'desarrollo', u'EdTech', u'quesepegue', u'Jacotot'])
(u'http://www.elmundo.es/espana/2017/01/23/58850e12268e3ebe398b45af.html', 410)
set([u'